# Constraints Learning

Assume you have an optimization problem written in terms of $\mathbf{\theta}$:

$\min_{\mathbf{\theta}} f(\mathbf{\theta})$

where $f$ maybe be for instance a polynomial or a rationial function, and $\theta \in \mathbb{R}^N$ may be multidimensional. We assume that you can write the above problem in an quivalent QCQP form by using a "lifting function" $\mathbf{l}(\theta) \in \mathbb{R}^M$ and defining the hihger-dimensional lifted state vector

$\mathbf{x}(\theta) = \begin{bmatrix}1 \\ \theta \\ z_1 \\ \vdots \\ z_M \end{bmatrix} = 
\begin{bmatrix}1 \\ \theta \\ \mathbf{l}(\theta) \end{bmatrix} \in \mathbb{R}^{1+N+M}$ 

Now we assume that each of the added constraints can itself be written as a quadratic function: 

$l_m(\theta) - z_m = \mathbf{x}(\theta)^\top \mathbf{A}_m \mathbf{x}(\theta) = 0$

where $\mathbf{A}_m$ ($m=1\ldots M$) are the constraints matrices. 
Sometimes, there may also exist redundant constraints, meaning some other matrices such that

$\mathbf{x}(\theta)^\top \mathbf{B}_m \mathbf{x}(\theta) = 0$. 

The goal of this not is to, for a given lifting function $\mathbf{l}(\theta)$, find the form of the redundant vs. primal constraints. 

**note that currently we just find all constraints and don't distinguish between primal (moment) constraints and redundant constraints**

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from IPython.display import display
%reload_ext autoreload
%autoreload 2

import shutil
usetex = True if shutil.which('latex') else False
print("found latex:", usetex)
plt.rcParams.update({
    "text.usetex": usetex,
    "font.family": "DejaVu Sans",
    "font.size": 12,
})
plt.rc('text.latex', preamble=r'\usepackage{bm}')
figsize = 7

#%matplotlib notebook
%matplotlib inline

from lifters.plotting_tools import savefig



# 0. Generate lifting functions

Currently implemented setups:
- Poly4Lifter 
- Poly6Lifter 
- RangeOnlyLocLifter  
- PoseLandmarkLifter 
- Stereo1DLifter 
- Stereo2DLifter 
- Stereo3DLifter 

In [ ]:
save = 1
#lifter_type = "poly"
#lifter_type = "rangeloc" 
#lifter_type = "rangeslam1" 
lifter_type = "rangeslam2" 
#lifter_type = "stereo"
d = 2
level = "urT" # see StereoLifter.LEVELS for possible Lasserre levels.
n_landmarks = 3
n_poses = 4

In [ ]:
from lifters.poly_lifters import Poly4Lifter, Poly6Lifter
from lifters.range_only_lifters import RangeOnlyLocLifter, RangeOnlySLAM1Lifter, RangeOnlySLAM2Lifter
from lifters.stereo1d_lifter import Stereo1DLifter
from lifters.stereo2d_lifter import Stereo2DLifter
from lifters.stereo3d_lifter import Stereo3DLifter
lifter = None
if lifter_type == "rangeloc":
    lifter = RangeOnlyLocLifter(n_landmarks=n_landmarks, n_positions=n_poses, d=d)
elif lifter_type == "rangeslam1":
    lifter = RangeOnlySLAM1Lifter(n_landmarks=n_landmarks, n_positions=n_poses, d=d)
elif lifter_type == "rangeslam2":
    lifter = RangeOnlySLAM2Lifter(n_landmarks=n_landmarks, n_positions=n_poses, d=d)
elif lifter_type == "poly":
    if d == 4:
        lifter = Poly4Lifter()
    elif d == 6:
        lifter = Poly6Lifter()
# stereo examples, working
elif lifter_type == "stereo":
    if d == 1:
        lifter = Stereo1DLifter(n_landmarks=n_landmarks)
    elif d == 2:
        lifter = Stereo2DLifter(n_landmarks=n_landmarks, level=level)
    elif d == 3:
        lifter = Stereo3DLifter(n_landmarks=n_landmarks, level=level)
if lifter is None:
    raise ValueError(lifter_type, level, d)
#from lifters.landmark_lifter import PoseLandmarkLifter
# just finds the moment constraint:
#lifter = PoseLandmarkLifter(n_landmarks=2, n_poses=1, d=2)

# 1. "learn" constraints matrices

The idea is to learn the nullspace of the matrix composed of many randomly generated feasible and lifted points.

In [ ]:
t = lifter.get_theta()
x = lifter.get_x(t)
print("x shape", x.shape)

In [ ]:
# generate many random setups and collect in matrix Y
Y = lifter.generate_Y(factor=2)
print("shape of setup matrix Y:", Y.shape)

In [ ]:
from lifters.plotting_tools import plot_singular_values
# compute nullspace of Y

method = "qr"
eps = 1e-4
basis, S = lifter.get_basis(Y, method=method, eps=eps)
print("nullspace basis:", basis.shape)

fig, ax = plot_singular_values(S, eps)
#if save:
#    savefig(fig, f"../_plots/svd_{lifter}.png")

In [ ]:
A_known = lifter.get_A_known()
for Ai in A_known:
    x = lifter.get_x(lifter.get_theta())
    assert abs(x.T @ Ai @ x) <= 1e-10

In [ ]:
# generate matrices from found nullspace

eps = 1e-5
A_list = lifter.generate_matrices(basis)

max_error = -np.inf

# testing only:
# make sure all constraints hold for new setups 
from lifters.stereo_lifter import get_theta_from_unknowns

for seed in range(1000):
    np.random.seed(seed)
    unknowns = lifter.sample_feasible(replace=False)
    x = lifter.get_x(unknowns)
    
    for i, A in enumerate(A_list):
        ci = np.abs(x.T @ A @ x)
        max_error = max(max_error, ci)
        #if seed == 0:
            #print(f"error of matrix {i}: {ci:.1e}")
        #if ci > eps:
        #    print(f"!! big error for seed {seed}, matrix {i}: {ci:.1e}")
print("max constraint error:", max_error)

In [ ]:
from lifters.plotting_tools import partial_plot_and_save
Q, y = lifter.get_Q()
partial_plot_and_save(lifter, Q, A_list, save=False)

# 2. Solve dual problem

Using the learned matrices, we solve the dual problem

$
\begin{align} 
d_n^* = &\max_{\rho, \mathbf{\lambda}} -\rho \\
&\text{s.t. } \mathbf{Q} + \sum_{m=1}^n \lambda_m \mathbf{A}_m + \rho \mathbf{A}_0 \succeq 0
\end{align}
$

where $n \leq N_0$ denotes the number of constraints we are adding
and compare the obtained cost to the cost of the (hopefully globally optimal) solution obtain by solving the original problem with a simple local solver:

$
\begin{align}
q^* &= \min_{\mathbf{\theta}} f(\mathbf{\theta})
\end{align}
$

In [ ]:
#df = pd.read_pickle("../_results/study_stereo1d_zero_noise.pkl")
import sys
sys.path.append("../_scripts/")
from noise_study import run_noise_study

params = dict(noise=1e-1, n_seeds=1, n_shuffles=0)
df = run_noise_study(lifter, A_list, **params)
#display(df)
#name = "study_stereo3d_mediumnoise_3level"
#df = pd.read_pickle(f"../_results/{name}.pkl")
df

In [ ]:
from lifters.plotting_tools import plot_tightness
fig, ax = plot_tightness(df)
if save:
    savefig(fig, f"../_plots/tightness_{lifter}.png")

In [ ]:
n_eigs = 5
palette = "viridis"
for seed, df_seed in df.groupby("seed"):
    fig, ax = plt.subplots()
    cmap = plt.get_cmap(lut=len(df_seed.n.unique()), name=palette)
    df_seed = df_seed[df_seed.shuffle==0]
    for (n, shuffle), df_n in df_seed.groupby(["n", "shuffle"]):
        assert len(df_n) == 1
        row = df_n.iloc[0]
        label = f"n={row.n}"
        if row.eigs is not None:
            ax.plot(row.eigs[:n_eigs], color=cmap(n), label=label)
            label = None
    ax.set_title(f"smallest {n_eigs} eigenvalues of H")
    ax.set_xticks(range(n_eigs))
    #ax.legend()
    ax.set_yscale("symlog")
    break

#  WIP: use sampling certificate

as proposed by Cifuentes & Parillo

In [ ]:
from lifters.stereo1d_lifter import Stereo1DLifter
from lifters.poly_lifters import Poly4Lifter
#n_landmarks = 3
#lifter = Stereo1DLifter(n_landmarks=n_landmarks)
lifter = Poly4Lifter()

In [ ]:
# initial guess of number of samples

U0_old = None
for S in range(lifter.dim_x, lifter.dim_x**2 + 10):
    print("number of samples:", S)

    # get orthogonal basis
    data = []
    samples = []
    for i in range(S):
        t = lifter.sample_feasible()
        samples.append(t)
        x = lifter.get_x(t)
        data.append(x[:, None])
    Umat = np.concatenate(data, axis=1)
    print(Umat.shape)


    U,s,U0 = np.linalg.svd(Umat, full_matrices=False)
    print("singular values:", s)

    T = U@np.diag(s)
    np.testing.assert_allclose(Umat, T@U0)

    Uhat = np.concatenate([np.outer(U0[:, i], U0[:, i]).flatten()[:, None] for i in range(U0.shape[1])], axis=1)
    assert Uhat.shape[1] == U0.shape[1]

    rank = np.linalg.matrix_rank(Uhat)
    print("rank of Uhat:", rank, " //  shape of Uhat:", Uhat.shape)
    if rank < Uhat.shape[1]:
        print("Uhat not full column rank: found enough samples")
        U0 = U0_old
        S = S-1
        sampels = samples[:-1]
        print(f"S={S}")
        break
    U0_old = U0

In [ ]:
from lifters.plotting_tools import plot_matrices

print(U0.shape)

A_list = [np.outer(U0[:, i], U0[:, i]) for i in range(U0.shape[1])]
fig, ax = plot_matrices(A_list=A_list, colorbar=False, nticks=3, vmin=-1, vmax=1)

In [ ]:
import cvxpy as cp 
from solvers.common import solver_options
solver = "CVXOPT"
options = solver_options[solver]
options["verbose"] = False

def solve_dual_sampling(samples, A_list):
    n = lifter.dim_x
    Q = cp.Variable((n, n), symmetric=True)
    lamda = cp.Variable()
    #lamda = 10
    constraints = [
        lifter.get_cost(t) - lamda == cp.trace(Q @ A_list[i]) for i, t in enumerate(samples)
        #lifter.get_cost(t) - lamda == U0[:, i].T @ Q @ U0[:, i] for i, t in enumerate(samples)
    ]
    constraints += [Q >> 0]
    prob = cp.Problem(cp.Maximize(lamda), constraints)
    try:
        prob.solve(solver=solver, **options)
        return prob.status, Q.value, lamda.value, constraints
    except:
        return "failed", None, None, None
    
for i in range(1, len(samples)+1):
    status, Q, l, constraints = solve_dual_sampling(samples[:i], A_list[:i])
    
    if Q is not None:
        eigs = np.linalg.eigvalsh(Q)[:3]
        print(len(constraints), len(samples[:i]))
        dual_vars = np.array([c.dual_value for c in constraints[:-1]])
        samples_i = np.array(samples[:i]).flatten()
        print(samples_i.shape, dual_vars.shape)
        X = np.sum(samples_i * dual_vars, axis=0)
    else:
        eigs = None
    if l is not None and (l > 0):
        print(f"i={i}, {status}: {eigs}, {np.sqrt(l)}")
    else:
        print(f"Warning: l negative")
        print(f"i={i}, {status}: {eigs}, {l}")
print(X)

In [ ]:
print("optimal Q:", Q.value, np.linalg.eigvalsh(Q.value))

# Example from paper

In [ ]:
At = np.r_[
np.c_[0.2190, 0.3835, 0.5297, 0.4175],
np.c_[0.0470, 0.5194, 0.6711, 0.6868], 
np.c_[0.6789, 0.8310, 0.0077, 0.5890], 
np.c_[0.6793, 0.0346, 0.3834, 0.9304], 
np.c_[0.9347, 0.0535, 0.0668, 0.8462]
].T
Bt = np.r_[
    np.c_[0.6942, 0.6526, 0.8299],
    np.c_[0.2110, 0.2204, 1.1734], 
    np.c_[0.2229, 0.2015, -0.1727],
    np.c_[-0.4104, 0.2994, 0.0474],
    np.c_[-0.9381, 1.0943, -0.2351],
].T
C = np.eye(3)
n, k, m1, m2 = 4, 3, 5, 3

u = lambda X: np.array([1, X.flatten()])
print("A", At.T.shape, "AT=", At)
print("B", Bt.T.shape, "BT=",Bt)

In [ ]:
def sample_feasible(n, k):
    A = np.random.rand(n, k)
    U, S, Vh = np.linalg.svd(A, full_matrices=False)
    V = U
    np.testing.assert_allclose(V.T @ V, np.eye(k), atol=1e-10)
    assert V.shape == (n, k), V.shape
    return V

S_min = n * k
for S in range(S_min, S_min**2):
    data = []
    samples = []
    for i in range(S):
        Xs = sample_feasible(n, k)
        samples.append(Xs)
        data.append(np.r_[1,  Xs.flatten()][:, None])
    Umat = np.concatenate(data, axis=1)
    
    U,s,U0 = np.linalg.svd(Umat, full_matrices=False)
    Uhat = np.concatenate([np.outer(U0[:, i], U0[:, i]).flatten()[:, None] for i in range(U0.shape[1])], axis=1)
    assert Uhat.shape[1] == U0.shape[1]

    rank = np.linalg.matrix_rank(Uhat)
    #print("rank of Uhat:", rank, " //  shape of Uhat:", Uhat.shape)
    if rank < Uhat.shape[1]:
        print("Uhat not full column rank: found enough samples")
        print(f"S={S}")
        break

In [ ]:
from lifters.plotting_tools import plot_matrices
print("U0 shape", U0.shape)

print(len(samples))
A_list = [np.outer(U0[:, i], U0[:, i]) for i in range(U0.shape[1])]
fig, ax = plot_matrices(A_list=A_list, colorbar=False, nticks=3, vmin=-1, vmax=1)

In [ ]:
import cvxpy as cp 
from solvers.common import solver_options
solver = "CVXOPT"
options = solver_options[solver]
print(options)
options["feastol"] = 1e-7
options["abstol"] = 1e-8
options["reltol"] = 1e-8
options["verbose"] = False

print(U0.shape)
print(At.T.shape, samples[0].shape, C.shape, Bt.T.shape)

def solve_sampled_sdp(samples, A_list):
    Q = cp.Variable((U0.shape[0], U0.shape[0]), symmetric=True)
    lamda = cp.Variable()
    constraints = [
        cp.norm(At.T @ samples[i] @ C - Bt.T, "fro")**2 - lamda == cp.trace(Q @ A_list[i]) for i, t in enumerate(samples)
        #lifter.get_cost(t) - lamda == U0[:, i].T @ Q @ U0[:, i] for i, t in enumerate(samples)
    ]
    constraints += [Q >> 0]
    print(len(constraints))
    prob = cp.Problem(cp.Maximize(lamda), constraints)
    try:
        prob.solve(solver=solver, **options)
    except Exception as e:
        return "failed", None, None, None
    return prob.status, Q.value, lamda.value, constraints
    
for i in range(1, len(samples))[-10:]:
    status, Q, l, constraints = solve_sampled_sdp(samples[:i], A_list[:i])
    if Q is not None:
        eigs = np.linalg.eigvalsh(Q)[:3]
        print(len(constraints), len(samples[:i]))
        dual_vars = np.array([c.dual_value for c in constraints[:-1]])
        samples_i = np.array(samples[:i])
        X = np.sum(samples_i * dual_vars[:, None, None], axis=0)
    else:
        eigs = None
    if l is not None and (l > 0):
        print(f"i={i}, {status}: {eigs}, {np.sqrt(l)}")
    else:
        print(f"Warning: l negative")
        print(f"i={i}, {status}: {eigs}, {l}")
print(X.T @ X)
print(-X.T)

# Next steps

- Using the $A_m$ that we know, try to complete the basis, so that it stays sparse. 